<a href="https://colab.research.google.com/github/GerardoMunoz/PresentacionesAlgebraLineal/blob/main/cos_word_vect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En la página https://matematicaj.blogspot.com/2020/03/Test-de-analogias-verbales-resuelto-con-claves-y-respuestas-pre-universidad-pdf.html se presentan algunas analogías de palabras. En el primer caso se pregunta soldado es a ejército como:
1. Ratón a gato,
2. Llave a llavero,
3. Paradoja a contradicción,
4. Pulga a perro,
5. Tornillo a máquina.

Usaremos el álgebra lineal para resolver el problema. Algoritmos (como bolsa de palabras, Word2vec, Gensim, GloVe) aplicado a un conjunto de documentos (corpus), [incrustan las palabras](https://unipython.com/como-desarrollar-embeddings-incrustaciones-de-palabras-con-gensim/) <!--(https://es.wikipedia.org/wiki/Word_embedding)--> en $\mathbb{R}^n$.  

Algunas de estas incrustaciones tienen propiedades semánticas, es decir que algunas operaciones vectoriales son congruentes con el significado de las palabras. Como en el famoso ejemplo  Rey - Hombre + Mujer -> Reina. 

Para este ejercicio vamos a utilizar la librería `spacy` con una base de datos de noticias (news) pequeña (sm) en español (es) llamada `es_core_news_sm`. 

* Lo primero es obtener el vector $\vec{u}_0$  para la palabra soldado y el vector $\vec{v}_0$ para la palabra ejército. Los vectores no están ni en $\mathbb{R}^2$ ni en $\mathbb{R}^3$ sino en $\mathbb{R}^{96}$

* Luego se obtiene el vector  $\vec{w}_0 =  \vec{u}_0 - \vec{v}_0$ que va desde soldado hasta ejército.

* De manera similar se obtiene el vector para la primera repuesta. Es decir, el vector $\vec{w}_1$ que va de raton a gato.  Finalmente se calcula el coseno del ángulo $\cos(\alpha_1)$ entre el vector  $\vec{w}_0$ y $\vec{w}_1$.

* Se repite lo mismo para cada par de palabras, obteniendo los vectores $\vec{w}_2$, $\vec{w}_3$, $\vec{w}_4$, $\vec{w}_5$. Posteriormente los cosenos  $\cos(\alpha_2)$, $\cos(\alpha_3)$, $\cos(\alpha_4)$, $\cos(\alpha_5)$, entre los vectores  $\vec{w}_0$ y el respectivo $\vec{w}_i$.

*  Finalmente se selecciona la respuesta con el mayor coseno del ángulo.

Lo primero es descargar la librería en español "es_core_news_sm". Esto sólo hay que realizarlo una vez se inicia. 



In [30]:
!python -m spacy download es_core_news_sm
# !python -m spacy download es_core_news_sm

2022-09-08 09:45:27.181137: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


En Colab puede 'Reinicar el entorno de ejecución' despues de descargar la librería 

Lo primero es llamar la libreria spacy. 

In [31]:
import spacy

Luego sigue cargar los datos del español que se descargaron previamente. En caso de no haber reiniciado el entorno de ejecución puede que se presente un error.

In [32]:
spacy_es = spacy.load("es_core_news_sm")

Ahora si comenzamos a obtener los vectores de cada palabra. Es importante que la palabra se escriba con tilde si las tiene. 

In [33]:
soldado=spacy_es("soldado").vector
soldado

array([-0.7155695 ,  1.3869509 , -1.8909149 ,  2.1056552 ,  1.5098524 ,
       -0.11164878, -2.0840092 ,  1.2565432 ,  3.1206698 , -1.3746428 ,
       -0.957276  , -0.9998431 ,  2.8895476 , -2.466204  , -2.0664263 ,
       -0.5012297 ,  2.025004  ,  1.6525549 ,  1.4513029 ,  2.5428417 ,
       -0.16929209, -0.43065274,  1.2244691 , -1.0512264 , -3.7376509 ,
        0.27136993, -2.4219816 , -0.14947842, -1.1031051 , -1.5842819 ,
        1.1171935 ,  0.12434453,  0.21441936, -0.4076234 , -0.40748632,
        4.508094  , -2.810264  ,  1.1510806 ,  0.37764937,  1.0818181 ,
        3.8766077 ,  2.246487  ,  0.7366568 , -0.09284037,  2.5397549 ,
        1.1426295 ,  1.4252193 ,  0.39980197,  1.9001868 , -2.280634  ,
       -2.3254886 , -0.06699443, -2.9225051 , -0.7963351 , -0.4020986 ,
        0.49715233,  1.2857593 , -0.32094732,  1.4612042 ,  3.0397143 ,
       -1.4021791 , -0.07928163,  0.22956175, -2.4282696 , -2.0647438 ,
        1.1913693 ,  0.86693513, -1.5781258 , -1.1418155 , -1.05

Observe que cada palabra es un vector de 96 elementos

In [34]:
len(soldado)

96

In [35]:
ejercito=spacy_es("ejército").vector
ejercito

array([ 1.3402033 ,  0.6466939 , -0.3120802 ,  2.0070653 , -1.4393866 ,
       -2.6502616 , -0.5631783 , -0.00893059,  4.530635  ,  0.6331577 ,
       -1.0969263 , -0.04058801,  0.03386638, -2.6523893 ,  4.486824  ,
       -0.28968263, -0.14905018, -0.79997635, -0.8146738 , -0.70976806,
       -0.23961245,  1.4245209 , -1.0063804 ,  2.9148288 , -2.3527927 ,
       -0.77206147, -3.5309649 ,  1.211537  ,  1.322429  , -0.28472412,
        1.2074524 ,  0.45431542, -1.756801  , -1.0503345 ,  0.46573648,
        2.1758225 ,  0.6006074 ,  0.52682555,  1.8131579 ,  0.9927753 ,
       -1.3028895 ,  1.156214  ,  3.0240312 , -0.84150076,  3.5365415 ,
        0.97008353, -1.0977478 , -1.0917639 ,  0.8708924 , -3.3688734 ,
       -0.07567406, -1.5869489 , -3.3947382 ,  3.8012545 , -1.4858091 ,
       -0.09389277, -1.8408613 , -0.30343014,  1.5598805 ,  1.162394  ,
       -0.7776882 ,  3.4819794 ,  3.0110652 , -0.12613082, -2.8761837 ,
       -0.8286469 , -1.0731697 , -3.5762599 , -0.21072918, -0.61

In [36]:
soldado_ejercito = soldado - ejercito
soldado_ejercito

array([-2.0557728 ,  0.74025697, -1.5788348 ,  0.0985899 ,  2.949239  ,
        2.5386128 , -1.5208309 ,  1.2654737 , -1.409965  , -2.0078006 ,
        0.13965034, -0.9592551 ,  2.8556812 ,  0.18618536, -6.5532503 ,
       -0.21154708,  2.1740541 ,  2.4525313 ,  2.2659767 ,  3.2526097 ,
        0.07032035, -1.8551736 ,  2.2308495 , -3.9660552 , -1.3848581 ,
        1.0434314 ,  1.1089833 , -1.3610154 , -2.425534  , -1.2995578 ,
       -0.09025896, -0.3299709 ,  1.9712204 ,  0.64271104, -0.8732228 ,
        2.3322713 , -3.4108715 ,  0.62425506, -1.4355085 ,  0.08904278,
        5.1794972 ,  1.0902729 , -2.2873745 ,  0.7486604 , -0.9967866 ,
        0.17254597,  2.522967  ,  1.4915658 ,  1.0292944 ,  1.0882394 ,
       -2.2498145 ,  1.5199544 ,  0.47223306, -4.5975895 ,  1.0837104 ,
        0.5910451 ,  3.1266208 , -0.01751718, -0.09867632,  1.8773203 ,
       -0.6244909 , -3.561261  , -2.7815034 , -2.3021388 ,  0.81144   ,
        2.0200162 ,  1.9401048 ,  1.998134  , -0.93108636, -0.43

In [37]:
raton = spacy_es("ratón").vector
gato = spacy_es("gato").vector
raton_gato = raton - gato
raton_gato

array([-0.80736995,  3.203731  ,  3.2945247 , -3.0995286 , -1.7742535 ,
       -2.6116729 ,  0.37929228,  1.1788265 ,  0.18508577,  0.42931688,
        0.5551119 ,  0.86605304,  0.27436614, -2.0092566 ,  0.45674708,
        1.3831381 , -0.45560932, -2.485649  ,  1.9864221 ,  2.8597052 ,
        2.970308  ,  0.17539611, -1.4346998 , -0.15388632,  1.8848097 ,
        0.31161022,  2.6513128 , -3.3837824 ,  3.9159436 ,  0.89486897,
        2.0937598 , -3.795614  , -3.3221571 ,  0.19060636,  1.7730912 ,
        0.56679285, -1.9354303 , -0.7166382 ,  1.1220381 ,  1.6204984 ,
        1.1352187 ,  0.31081188, -2.1341548 , -0.24558914, -0.80052733,
        0.8841469 , -1.3709617 ,  0.08220857,  1.1576394 ,  3.3177917 ,
        2.5158405 ,  0.5931077 ,  3.0666013 ,  1.3191019 , -2.0501442 ,
        5.4901457 , -1.2465388 ,  3.583285  ,  2.0075166 ,  2.8052423 ,
       -0.9255254 ,  0.10486704,  0.05997324,  0.28149176,  0.95316863,
       -0.44401443, -0.58193856, -1.6338973 , -0.31109917, -4.86

In [38]:
import numpy as np
# Los vectores obtenidos son arreglos de numpy
# El producto punto entre dos vectores se puede realizar con el método `dot`
# La norma de un vector se realiza con la función `np.linalg.norm`

In [39]:
cos_preg_rta1 = soldado_ejercito.dot(raton_gato) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(raton_gato))
cos_preg_rta1

0.005233307

In [40]:
llave = spacy_es("llave").vector
llavero = spacy_es("llavero").vector
llave_llavero = llave - llavero

cos_preg_rta2 = soldado_ejercito.dot(llave_llavero) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(llave_llavero))
cos_preg_rta2

0.08419277

In [41]:
paradoja = spacy_es("paradoja").vector
contradiccion = spacy_es("contradicción").vector
paradoja_contradiccion = paradoja - contradiccion

cos_preg_rta3 = soldado_ejercito.dot(paradoja_contradiccion) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(paradoja_contradiccion))
cos_preg_rta3

-0.17580466

In [42]:
pulga = spacy_es("pulga").vector
perro = spacy_es("perro").vector
pulga_perro = pulga - perro

cos_preg_rta4 = soldado_ejercito.dot(pulga_perro) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(pulga_perro))
cos_preg_rta4

-0.23316221

In [43]:
tornillo = spacy_es("tornillo").vector
maquina = spacy_es("máquina").vector
tornillo_maquina = tornillo - maquina

cos_preg_rta5 = soldado_ejercito.dot(tornillo_maquina) / (np.linalg.norm(soldado_ejercito) * np.linalg.norm(tornillo_maquina))
cos_preg_rta5

0.19560717



La mejor respuesta es la que el valor absoluto del coseno del ángulo sea mayor. 
